In [1]:
import pandas as pd
import torch as th
import torch.nn as nn
import glob
import pickle

import sys
sys.path.append('../models/')
from models.lstm import LSTM
from models.transformer import Transformer
from models.train_model import RPS_loss, train_model

In [2]:
training_home_teams_matches = th.load("dataset/tensors/training_home_teams_matches.pt")
training_away_teams_matches = th.load("dataset/tensors/training_away_teams_matches.pt")
training_matches_features_home = th.load("dataset/tensors/training_matches_features_home.pt")
training_matches_features_away = th.load("dataset/tensors/training_matches_features_away.pt")
training_targets = th.load("dataset/tensors/training_targets.pt")

test_home_teams_matches = th.load("dataset/tensors/test_home_teams_matches.pt")
test_away_teams_matches = th.load("dataset/tensors/test_away_teams_matches.pt")
test_matches_features_home = th.load("dataset/tensors/test_matches_features_home.pt")
test_matches_features_away = th.load("dataset/tensors/test_matches_features_away.pt")
test_targets = th.load("dataset/tensors/test_targets.pt")



In [3]:
import pickle
file1 = 'dataset/tensors/idx_to_teams.pkl'
file2 = 'dataset/tensors/teams_to_idx.pkl'
file3 = 'dataset/tensors/result_map.pkl'

with open(file1, 'rb') as file:
    idx_to_teams = pickle.load(file)

with open(file2, 'rb') as file:
    teams_to_idx = pickle.load(file)

with open(file3, 'rb') as file:
    result_map = pickle.load(file)

print(result_map)
idx_to_result = {0: 'H', 1: 'D', 2: 'A'}



{'H': 0, 'D': 1, 'A': 2}


In [4]:
print("Training data")
print(training_home_teams_matches.shape) # the idex of home team for the matches
print(training_away_teams_matches.shape) # the idex of away team for the matches
print(training_matches_features_home.shape) # the features of the home team for the matches
print(training_matches_features_away.shape) # the features of the away team for the matches
print(training_targets.shape) # the targets of the matches H D A

print("Test data")
print(test_home_teams_matches.shape)
print(test_away_teams_matches.shape)
print(test_matches_features_home.shape)
print(test_matches_features_away.shape)
print(test_targets.shape)




Training data
torch.Size([24843])
torch.Size([24843])
torch.Size([24843, 5, 14])
torch.Size([24843, 5, 14])
torch.Size([24843])
Test data
torch.Size([3072])
torch.Size([3072])
torch.Size([3072, 5, 14])
torch.Size([3072, 5, 14])
torch.Size([3072])


In [5]:
import torch.nn.functional as F

def one_hot_targets(targets):
    num_classes = th.max(targets).item() + 1
    return F.one_hot(targets, num_classes=num_classes).type(th.float32)

training_targets_oh = one_hot_targets(training_targets)
test_targets_oh = one_hot_targets(test_targets)

print(training_targets_oh.shape)
print(test_targets_oh.shape)

for i in range(3):
    print(training_targets_oh[i])
    print(test_targets_oh[i])

torch.Size([24843, 3])
torch.Size([3072, 3])
tensor([0., 1., 0.])
tensor([0., 0., 1.])
tensor([1., 0., 0.])
tensor([0., 1., 0.])
tensor([0., 0., 1.])
tensor([1., 0., 0.])


In [6]:
index = 0
print("First match")
print("Home team: ", idx_to_teams[training_home_teams_matches[index].item()])
print("Away team: ", idx_to_teams[training_away_teams_matches[index].item()])
print("Features home: ", training_matches_features_home[index])
print("Features away: ", training_matches_features_away[index])
# print("Target: ", idx_to_result[training_targets[index].item()])


First match
Home team:  Bourg Peronnas
Away team:  Laval
Features home:  tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  2.7000,  2.8800,  2.9000],
        [-1.1730, -1.8492, -0.8932, -0.3694,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  2.2500,  3.0000,  3.5000],
        [-0.6845,  1.3282, -1.1527,  0.8992,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  2.4000,  3.0000,  3.2000],
        [-0.2291,  1.8941, -1.4227,  0.5266,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  3.1000,  3.1000,  2.3800],
        [ 0.2685,  1.2267, -1.1430, -0.6323,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  2.1000,  3.1000,  3.7500]])
Features away:  tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  2.2000,  3.0000,  3.6000],
        [-0.3644, -0.6365, -0.8932, -1.1

In [7]:
#split the data into training and validation
from torch.utils.data import DataLoader, TensorDataset

split = 0.8
split_idx = int(len(training_home_teams_matches) * split)

train_dataset = TensorDataset(training_home_teams_matches[:split_idx], training_away_teams_matches[:split_idx], training_matches_features_home[:split_idx], training_matches_features_away[:split_idx], training_targets_oh[:split_idx])
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False) # true could cause lookahead

val_dataset = TensorDataset(training_home_teams_matches[split_idx:], training_away_teams_matches[split_idx:], training_matches_features_home[split_idx:], training_matches_features_away[split_idx:], training_targets_oh[split_idx:])
# val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

test_dataset = TensorDataset(test_home_teams_matches, test_away_teams_matches, test_matches_features_home, test_matches_features_away, test_targets_oh)
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print("Training data")
print(len(train_dataset))
# print(len(train_loader))

print("Validation data")
print(len(val_dataset))
# print(len(val_loader))

print("Test data")
print(len(test_dataset))
# print(len(test_loader))

Training data
19874
Validation data
4969
Test data
3072


In [ ]:
model_lstm = LSTM(
            num_features=14,
            inception_depth=2, 
            inception_out=24, 
            lstm_num_layers=2,
            hidden_size=500, 
            num_heads=1, num_classes=3, bottleneck_dim = None)

# Default batch size = 100

train_model(model_lstm, 
criterion=RPS_loss(),
# criterion= nn.CrossEntropyLoss(),
train_dataset=train_dataset, 
val_dataset=val_dataset,
learning_rate=0.1,
num_epochs=10)

In [10]:
model_transformer = Transformer(
            num_features=14,
            inception_depth=2, 
            inception_out=24,
            num_heads=8,
            num_layers=6,
            d_ff = 256,
            d_h = 512,
            max_timespan = 5, # axis (1) of data
            dropout = 0.1)

train_model(model_transformer, 
# criterion=RPS_loss(),
criterion= nn.CrossEntropyLoss(),
train_dataset=train_dataset, 
val_dataset=val_dataset,
learning_rate=0.00001,
num_epochs=1000)

50 Loss: 0.11096549034118652 Train Acc: 38.92964824120603 Val Acc: 36.52
100 Loss: 0.10961967706680298 Train Acc: 41.29145728643216 Val Acc: 38.82
150 Loss: 0.11596894264221191 Train Acc: 43.32663316582914 Val Acc: 43.16
200 Loss: 0.11513856798410416 Train Acc: 43.41708542713568 Val Acc: 43.34
250 Loss: 0.1091860681772232 Train Acc: 43.70854271356784 Val Acc: 43.78
300 Loss: 0.10303571075201035 Train Acc: 43.49246231155779 Val Acc: 42.5
350 Loss: 0.1144317239522934 Train Acc: 44.1356783919598 Val Acc: 43.22
400 Loss: 0.11053590476512909 Train Acc: 44.49748743718593 Val Acc: 43.18
450 Loss: 0.10015374422073364 Train Acc: 44.80904522613066 Val Acc: 44.16
500 Loss: 0.11531666666269302 Train Acc: 45.20603015075377 Val Acc: 44.48
550 Loss: 0.10971035063266754 Train Acc: 45.36683417085427 Val Acc: 44.74
600 Loss: 0.10515064001083374 Train Acc: 45.58291457286432 Val Acc: 44.48
650 Loss: 0.11006437242031097 Train Acc: 46.06030150753769 Val Acc: 45.34
700 Loss: 0.1035480871796608 Train Acc: 46.